# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

Basic sanity-check notebook for Drop-In Knowledge Mode runtime.

Goals:
  - Verify that DropInKnowledgeRuntime can:
      * create or load a session,
      * append user and assistant messages,
      * build conversation history from SessionStore,
      * return a RuntimeAnswer object.
  - Use InMemorySessionStore for simplicity.
  - Keep LLM integration as placeholder for now.

Create config, session store, runtime

In [2]:
from datetime import datetime
from intergrax.llm_adapters.llm_provider import LLMProvider
from intergrax.llm_adapters.llm_provider_registry import LLMAdapterRegistry
from intergrax.runtime.drop_in_knowledge_mode.config import RuntimeConfig
from intergrax.runtime.drop_in_knowledge_mode.engine.runtime import DropInKnowledgeRuntime
from intergrax.runtime.drop_in_knowledge_mode.session.in_memory_session_storage import InMemorySessionStorage
from intergrax.runtime.drop_in_knowledge_mode.session.session_manager import SessionManager

# Instantiate the in-memory session store
session_manager = SessionManager(
    storage=InMemorySessionStorage()
)

# Instantiate a LLM adapter
llm_adapter = LLMAdapterRegistry.create(LLMProvider.OLLAMA)

TENANT = "intergrax"
CORPUS = "intergrax-strategy"
VERSION = "v1"

config = RuntimeConfig(
    llm_adapter=llm_adapter,
    enable_rag=False,        # disabled for this initial sanity check
    enable_websearch=False,  # disabled for this initial sanity check
    tools_mode="off",      # disabled for this initial sanity check
    enable_llm_usage_collection=True,
)

runtime = DropInKnowledgeRuntime(
    config=config,
    session_manager=session_manager,
)

print("Runtime initialized at", datetime.utcnow().isoformat())


Runtime initialized at 2025-12-26T10:15:46.885875


C:\Users\XPS\AppData\Local\Temp\ipykernel_10828\287699158.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print("Runtime initialized at", datetime.utcnow().isoformat())


First request: create a new session

In [3]:
# Define user and session identifiers
from intergrax.runtime.drop_in_knowledge_mode.responses.response_schema import RuntimeRequest


user_id = "demo-user-001"
session_id = "demo-session-001"

request_1 = RuntimeRequest(
    user_id=user_id,
    session_id=session_id,
    message="Hello Intergrax runtime, this is my first message.",
    metadata={"source": "notebook-demo"},
)

answer_1 = await runtime.run(request_1)

print("=== ANSWER 1 ===")
print(answer_1.answer)
print("\nRoute info:")
print(answer_1.route)
print("\nStats:")
print(answer_1.stats)
print("\nDebug trace:")
print(answer_1.debug_trace)


=== ANSWER 1 ===
Welcome to the Intergrax runtime! This is an exciting new platform for experimentation and innovation.

I'm here to help you explore the capabilities of the Intergrax runtime and support you in achieving your goals. What would you like to do or discuss? You can ask me any questions, share ideas, or let's just chat about what Intergrax has to offer!

Route info:
RouteInfo(used_rag=False, used_websearch=False, used_tools=False, used_user_profile=False, used_user_longterm_memory=False, strategy='llm_only', extra={})

Stats:
RuntimeStats(total_tokens=None, input_tokens=None, output_tokens=None, rag_tokens=None, websearch_tokens=None, tool_tokens=None, duration_ms=None, extra={})

Debug trace:
{'session_id': 'demo-session-001', 'user_id': 'demo-user-001', 'memory_layer': {'implemented': True, 'has_user_profile_instructions': False, 'has_org_profile_instructions': False, 'enable_user_profile_memory': True, 'enable_org_profile_memory': True}, 'steps': [{'timestamp': '2025-12-

Inspect session state after first request

In [4]:
session = await session_manager.get_session(answer_1.debug_trace["session_id"])
messages = await session_manager.get_history(session_id=answer_1.debug_trace["session_id"])

print("=== SESSION AFTER FIRST REQUEST ===")
print("Session ID:", session.id)
print("User ID:", session.user_id)
print("Messages count:", len(messages))
print("Created at:", session.created_at)
print("Updated at:", session.updated_at)

print("\nMessages:")
for idx, msg in enumerate(messages, start=1):
    print(f"  [{idx}] role={msg.role!r}, created_at={msg.created_at}")
    print(f"      content={msg.content!r}")


=== SESSION AFTER FIRST REQUEST ===
Session ID: demo-session-001
User ID: demo-user-001
Messages count: 2
Created at: 2025-12-26 10:15:46.899639+00:00
Updated at: 2025-12-26 10:15:49.172904+00:00

Messages:
  [1] role='user', created_at=2025-12-26T10:15:46.899639+00:00
      content='Hello Intergrax runtime, this is my first message.'
  [2] role='assistant', created_at=2025-12-26T10:15:49.172904+00:00
      content="Welcome to the Intergrax runtime! This is an exciting new platform for experimentation and innovation.\n\nI'm here to help you explore the capabilities of the Intergrax runtime and support you in achieving your goals. What would you like to do or discuss? You can ask me any questions, share ideas, or let's just chat about what Intergrax has to offer!"


Second request: same session, history should grow

In [5]:
request_2 = RuntimeRequest(
    user_id=user_id,
    session_id=session_id,
    message="This is my second message in the same session.",
    metadata={"source": "notebook-demo"},
)

answer_2 = await runtime.run(request_2)

print("=== ANSWER 2 ===")
print(answer_2.answer)
print("\nRoute info:")
print(answer_2.route)
print("\nDebug trace:")
print(answer_2.debug_trace)

session_2 = await session_manager.get_session(answer_2.debug_trace["session_id"])
messages = await session_manager.get_history(session_id=answer_1.debug_trace["session_id"])

print("\n=== SESSION AFTER SECOND REQUEST ===")
print("Session ID:", session_2.id)
print("Messages count:", len(messages))
for idx, msg in enumerate(messages, start=1):
    print(f"  [{idx}] role={msg.role!r}, created_at={msg.created_at}")
    print(f"      content={msg.content!r}")


=== ANSWER 2 ===
You've sent a follow-up message. How are things progressing on the Intergrax runtime so far? Have you encountered any interesting developments or would you like some guidance on how to proceed? I'm here to help and provide support as needed. What's on your mind?

Route info:
RouteInfo(used_rag=False, used_websearch=False, used_tools=False, used_user_profile=False, used_user_longterm_memory=False, strategy='llm_only', extra={})

Debug trace:
{'session_id': 'demo-session-001', 'user_id': 'demo-user-001', 'memory_layer': {'implemented': True, 'has_user_profile_instructions': False, 'has_org_profile_instructions': False, 'enable_user_profile_memory': True, 'enable_org_profile_memory': True}, 'steps': [{'timestamp': '2025-12-26T10:15:49.188643+00:00', 'component': 'engine', 'step': 'memory_layer', 'message': 'Profile-based instructions loaded for session.', 'data': {'has_user_profile_instructions': False, 'has_org_profile_instructions': False, 'enable_user_profile_memory': 

Verify that history length is limited by config

In [6]:
from intergrax.llm.messages import ChatMessage

# For debugging: call the history builder directly
session_for_history = await session_manager.get_session(answer_2.debug_trace["session_id"])
history = await runtime._history_layer._build_chat_history(session_for_history)

print("=== BUILT CHAT HISTORY ===")
print("History length:", len(history))
for idx, msg in enumerate(history, start=1):
    assert isinstance(msg, ChatMessage)
    print(f"  [{idx}] role={msg.role!r}, created_at={msg.created_at}")
    print(f"      content={msg.content!r}")


=== BUILT CHAT HISTORY ===
History length: 4
  [1] role='user', created_at=2025-12-26T10:15:46.899639+00:00
      content='Hello Intergrax runtime, this is my first message.'
  [2] role='assistant', created_at=2025-12-26T10:15:49.172904+00:00
      content="Welcome to the Intergrax runtime! This is an exciting new platform for experimentation and innovation.\n\nI'm here to help you explore the capabilities of the Intergrax runtime and support you in achieving your goals. What would you like to do or discuss? You can ask me any questions, share ideas, or let's just chat about what Intergrax has to offer!"
  [3] role='user', created_at=2025-12-26T10:15:49.188643+00:00
      content='This is my second message in the same session.'
  [4] role='assistant', created_at=2025-12-26T10:15:50.700592+00:00
      content="You've sent a follow-up message. How are things progressing on the Intergrax runtime so far? Have you encountered any interesting developments or would you like some guidance on h

In [ ]:
runs = await runtime.get_llm_usage_runs()
print("runs:", len(runs))
for r in runs[-5:]:
    total = r.report.total    
    print(f"[{r.seq}] {r.ts_utc.isoformat()} run_id={r.run_id} calls={total.calls} total_tokens={total.total_tokens}")


runs: 2
[1] 2025-12-26T10:15:49.172904+00:00 run_id=run_73aaf58a15d948868cf7a430a68450dc calls=1 total_tokens=87
[2] 2025-12-26T10:15:50.700592+00:00 run_id=run_a4904699d0104c80a30a869f5a833664 calls=1 total_tokens=152
